In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
import xgboost as xgb
import numpy as np
import math

dataset = 'covertype'
seed = 10000
n = 300 # number of key cells
gamma = 1/2 # ratio between the length of green domain and red domain


# 读取数据
origin = pd.read_csv("../../../datasets/covertype/cover_type_with_columns.csv")

proportions = [0.2, 0.4, 0.6, 0.8, 1.0]

for proportion in proportions:
    loaded_results = np.load(f"../../../datasets/covertype/watermarked/{dataset}-{seed}.npy", allow_pickle=True).item()
    # watermarked_data = loaded_results['watermarked_data']
    watermarked_data = pd.read_csv(f"../../../datasets/covertype/alteration/{dataset}-{seed}-{proportion}-{0}.csv")
    divide_seeds = loaded_results['divide_seeds']
    indices = loaded_results['indices']


    #添加水印
    cover_types = watermarked_data['Cover_Type'].unique()
    cover_types.sort()  

    green_cell = 0
    for idx, divide_seed in zip(indices, divide_seeds):
        np.random.seed(divide_seed)
        candidate_set = cover_types
        # 打乱cover_types的顺序
        shuffled_cover_types = list(cover_types)
        # print(shuffled_cover_types)
        np.random.shuffle(shuffled_cover_types)

        # 确保cover_types能被划分为两个相等大小的部分
        half_size = len(shuffled_cover_types) // 2

        # 划分成green_domain和red_domain
        green_domain = shuffled_cover_types[:half_size]
        red_domain = shuffled_cover_types[half_size:]

        if watermarked_data.loc[idx, 'Cover_Type'] in green_domain:
            green_cell += 1
        
    z_score = (green_cell - n/2) / math.sqrt(n/4)
    print(f"{proportion}: The z-score is ", z_score)



0.2: The z-score is  14.202816622064793
0.4: The z-score is  9.699484522385712
0.6: The z-score is  5.888972745734182
0.8: The z-score is  1.5011106998930268
1.0: The z-score is  -0.5773502691896257
